### Middleware Basics

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
from langchain.chat_models import init_chat_model

model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=.5)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=.5)
model_gpt_4o_mini = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=.5)

### Summarize messages

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware



agent = create_agent(
    model=model_llama_groq,
    checkpointer=InMemorySaver(),
    middleware=[SummarizationMiddleware(
        model=model_llama_groq,
        trigger=("tokens", 100),
        keep=("messages",1)
    )]
)



In [ ]:
from langchain.messages import HumanMessage, AIMessage
response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

### Trim/delete messages

In [ ]:
from typing import Any
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent,AgentState
from langchain.messages import ToolMessage
from langgraph.checkpoint.memory import InMemorySaver

@before_agent
def trim_messages(state:AgentState, runtime:Runtime) -> dict[str,Any]|None:
    "Remove all the tool messages from the prompt"
    messages = state["messages"]
    tool_messages = [ m for m in messages if isinstance(m,ToolMessage)]
    return {"messages":[RemoveMessage(id=m.id) for m in tool_messages]}


agent = create_agent(
    model=model_llama_groq,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages]
)

response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)



In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

### HumanInTheLoop (HITL) middleware

In [ ]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model_gemini_flash,
    tools=[read_email, send_email],
    checkpointer=InMemorySaver(),
     state_schema=EmailState,
     middleware=[HumanInTheLoopMiddleware(
        interrupt_on={
            "send_email":True
        },
        description_prefix="Tool execution requires approval "
     )],
     system_prompt="You are email assistant. Make use of read_email tool to get the email content. Come up with a response without any followup questions for additional inputs and use send_email tool to send the email"
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "6"}}
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response['__interrupt__'])

In [ ]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

### Approve

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)



In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

### Reject

In [ ]:
from uuid import uuid4

print(uuid4())

In [ ]:
agent = create_agent(
    model=model_gpt_4o_mini,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[HumanInTheLoopMiddleware(
        interrupt_on={
            "send_email":True
        },
        description_prefix="Tool execution requires approval "
    )],
    system_prompt="You are email assistant. Make use of read_email tool to get the email content. Come up with a response without any followup questions for additional inputs and use send_email tool to send the email"

)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": uuid4()}}
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [ ]:
pprint(response)

In [ ]:
print(response['__interrupt__'])

In [ ]:
from langgraph.types import Command

response = agent.invoke(Command(
    resume={
        "decisions":[{"type":"reject","message":"No, please signoff - You merciful leader, Sean"}]
    }
), config=config
)

In [ ]:
pprint(response)


### Edit

In [ ]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

In [ ]:
for msg in response["messages"]:
    msg.pretty_print()